In [3]:
class BST:
    def __init__(self):
        self.root = None

    class Node:
        def __init__(self, key):
            self.key = key
            self.left = None
            self.right = None
            self.parent = None

    def insert(self, key):
        new_node = self.Node(key)
        if not self.root:
            self.root = new_node
            return
        node = self.root
        while node:
            parent = node
            if key < node.key:
                node = node.left
            else:
                node = node.right
        new_node.parent = parent
        if key < parent.key:
            parent.left = new_node
        else:
            parent.right = new_node

    def search(self, key):
        node = self.root
        while node and node.key != key:
            node = node.left if key < node.key else node.right
        return node

    def delete(self, key):
        node = self.search(key)
        if not node:
            return
        if not node.left or not node.right:
            self.transplant(node, node.left or node.right)
        else:
            successor = self.minimum(node.right)
            if successor.parent != node:
                self.transplant(successor, successor.right)
                successor.right = node.right
                successor.right.parent = successor
            self.transplant(node, successor)
            successor.left = node.left
            successor.left.parent = successor

    def transplant(self, u, v):
        if not u.parent:
            self.root = v
        elif u == u.parent.left:
            u.parent.left = v
        else:
            u.parent.right = v
        if v:
            v.parent = u.parent

    def minimum(self, node):
        while node.left:
            node = node.left
        return node

In [12]:
bst = BST()
print(f'Inserting 21, 9, 16.')
bst.insert(21)
bst.insert(9)
bst.insert(16)
print(f'Searching 9 | Found:',bst.search(9) is not None)
bst.delete(9)
print(f'Deleting 9')
print(f'Searching 9 | Found:',bst.search(9) is not None)

Inserting 21, 9, 16.
Searching 9 | Found: True
Deleting 9
Searching 9 | Found: False


In [17]:

class RedBlackTree(BST):
    class Node(BST.Node):
        def __init__(self, key, color='R'):
            super().__init__(key)
            self.color = color

    def insert(self, key):
        new_node = self.Node(key)
        super().insert(key)
        self.fix_insert(new_node)

    def fix_insert(self, node):
        while node.parent and node.parent.color == 'R':
            grandparent = node.parent.parent
            if node.parent == grandparent.left:
                uncle = grandparent.right
                if uncle and uncle.color == 'R':
                    node.parent.color = 'B'
                    uncle.color = 'B'
                    grandparent.color = 'R'
                    node = grandparent
                else:
                    if node == node.parent.right:
                        node = node.parent
                        self.rotate_left(node)
                    node.parent.color = 'B'
                    grandparent.color = 'R'
                    self.rotate_right(grandparent)
            else:
                uncle = grandparent.left
                if uncle and uncle.color == 'R':
                    node.parent.color = 'B'
                    uncle.color = 'B'
                    grandparent.color = 'R'
                    node = grandparent
                else:
                    if node == node.parent.left:
                        node = node.parent
                        self.rotate_right(node)
                    node.parent.color = 'B'
                    grandparent.color = 'R'
                    self.rotate_left(grandparent)
        self.root.color = 'B'

    def delete(self, key):
        node = self.search(key)
        if not node:
            return
        super().delete(key)
        self.fix_delete(node)

    def fix_delete(self, node):
        while node and node != self.root and node.color == 'B':
            if node.parent:
                sibling = node.parent.right if node == node.parent.left else node.parent.left
                if sibling and sibling.color == 'R':
                    sibling.color = 'B'
                    node.parent.color = 'R'
                    if node == node.parent.left:
                        self.rotate_left(node.parent)
                    else:
                        self.rotate_right(node.parent)
                    sibling = node.parent.right if node == node.parent.left else node.parent.left
                if sibling and sibling.left and sibling.right and sibling.left.color == 'B' and sibling.right.color == 'B':
                    sibling.color = 'R'
                    node = node.parent
                else:
                    if sibling:
                        if (node == node.parent.left and sibling.right and sibling.right.color == 'B') or \
                           (node == node.parent.right and sibling.left and sibling.left.color == 'B'):
                            sibling.color = 'R'
                            if node == node.parent.left and sibling.left:
                                sibling.left.color = 'B'
                                self.rotate_right(sibling)
                            elif node == node.parent.right and sibling.right:
                                sibling.right.color = 'B'
                                self.rotate_left(sibling)
                            sibling = node.parent.right if node == node.parent.left else node.parent.left
                        sibling.color = node.parent.color
                        node.parent.color = 'B'
                        if node == node.parent.left and sibling.right:
                            sibling.right.color = 'B'
                            self.rotate_left(node.parent)
                        elif node == node.parent.right and sibling.left:
                            sibling.left.color = 'B'
                            self.rotate_right(node.parent)
                    node = self.root
        if node:
            node.color = 'B'



In [19]:
rbt = RedBlackTree()
print(f'Inserting 21, 9, 16, 17, 21, 24, 20, 25.')
rbt.insert(21)
rbt.insert(9)
rbt.insert(16)
rbt.insert(17)
rbt.insert(21)
rbt.insert(24)
rbt.insert(20)
rbt.insert(25)

print(f'Searching 21 | Found:',rbt.search(21) is not None)
print(f'Searching 9 | Found:',rbt.search(9) is not None)

Inserting 21, 9, 16, 17, 21, 24, 20, 25.
Searching 21 | Found: True
Searching 9 | Found: True


In [22]:
class AVLTree(BST):
    def insert(self, key):
        super().insert(key)
        self.rebalance(self.search(key))

    def delete(self, key):
        super().delete(key)
        self.rebalance(self.search(key))

    def rebalance(self, node):
        while node:
            balance = self.get_balance(node)
            if balance > 1:
                if self.get_balance(node.left) < 0:
                    self.rotate_left(node.left)
                self.rotate_right(node)
            elif balance < -1:
                if self.get_balance(node.right) > 0:
                    self.rotate_right(node.right)
                self.rotate_left(node)
            node = node.parent

    def rotate_left(self, node):
        right_child = node.right
        if not right_child:
            return
        node.right = right_child.left
        if right_child.left:
            right_child.left.parent = node
        right_child.parent = node.parent
        if not node.parent:
            self.root = right_child
        elif node == node.parent.left:
            node.parent.left = right_child
        else:
            node.parent.right = right_child
        right_child.left = node
        node.parent = right_child

    def rotate_right(self, node):
        left_child = node.left
        if not left_child:
            return
        node.left = left_child.right
        if left_child.right:
            left_child.right.parent = node
        left_child.parent = node.parent
        if not node.parent:
            self.root = left_child
        elif node == node.parent.right:
            node.parent.right = left_child
        else:
            node.parent.left = left_child
        left_child.right = node
        node.parent = left_child

    def get_balance(self, node):
        return self.height(node.left) - self.height(node.right)

    def height(self, node):
        if not node:
            return 0
        return 1 + max(self.height(node.left), self.height(node.right))

In [27]:
avl = AVLTree()
print(f'Inserting 21, 9, 16, 17, 21, 24, 20, 25.')
avl.insert(21)
avl.insert(9)
avl.insert(16)
avl.insert(17)
avl.insert(21)
avl.insert(24)
avl.insert(20)
avl.insert(25)

print(f'Searching 21 | Found:',avl.search(21) is not None)
avl.delete(21)
print(f'Searching 9 | Found:',avl.search(9) is not None)

Inserting 21, 9, 16, 17, 21, 24, 20, 25.
Searching 21 | Found: True
Searching 9 | Found: True
